# Ausführen von Skripts als Pipelineauftrag

Mit einer Pipeline können Sie mehrere Schritte in einem Workflow gruppieren. Sie können eine Pipeline mit Komponenten erstellen. Jede Komponente spiegelt ein auszuführendes Python-Skript wider. Eine Komponente wird in einer YAML-Datei definiert, die das Skript und dessen Ausführung angibt. 

## Vorbereitung

Sie benötigen die neueste Version des Pakets **azureml-ai-ml**, um den Code in diesem Notebook auszuführen. Führen Sie die folgende Zelle aus, um zu überprüfen, ob das Paket installiert ist.

> **Hinweis**:
> Wenn das Paket **azure-ai-ml** nicht installiert ist, führen Sie `pip install azure-ai-ml` aus, um es zu installieren.

In [ ]:
## Herstellen einer Verbindung mit Ihrem Arbeitsbereich

Sie können nun eine Verbindung mit Ihrem Arbeitsbereich herstellen, nachdem Sie die erforderlichen SDK-Pakete installiert haben.

Um eine Verbindung mit einem Arbeitsbereich herzustellen, benötigen Sie Bezeichnerparameter: eine Abonnement-ID, einen Ressourcengruppennamen und einen Arbeitsbereichsnamen. Der Ressourcengruppenname und Arbeitsbereichsname sind bereits für Sie ausgefüllt. Sie müssen lediglich die Abonnement-ID angeben, um den Befehl zu vervollständigen.

Um die erforderlichen Parameter zu finden, klicken Sie rechts oben in Studio auf das Abonnement und den Namen des Arbeitsbereichs. Rechts wird ein Bereich geöffnet.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Kopieren Sie die Abonnement-ID, und ersetzen Sie **YOUR-SUBSCRIPTION-ID** durch den Wert, den Sie kopiert haben. </p>

## Erstellen der Skripts

Sie erstellen eine Pipeline in zwei Schritten:

1. **Aufbereiten der Daten**: Korrigieren Sie fehlende Daten, und normalisieren Sie die Daten.
1. **Trainieren des Modells**: Trainieren Sie ein Modell zur Entscheidungsstrukturklassifizierung.

Führen Sie die folgenden Zellen aus, um den Ordner **src** und die beiden Skripts zu erstellen.

In [ ]:
## Definieren der Komponenten

So definieren Sie die Komponente, die Sie angeben müssen

- **Metadaten**: *Name*, *Anzeigename*, *Version*, *Beschreibung*, *Typ* usw. Die Metadaten helfen, die Komponente zu beschreiben und zu verwalten.
- **Schnittstelle**: *Eingaben* und *Ausgaben*. Beispielsweise verwendet eine Modelltrainingskomponente Trainingsdaten und die Abgrenzungsrate als Eingabe und generiert eine trainierte Modelldatei als Ausgabe. 
- **Befehl, Code und Umgebung**: Der *Befehl*, *Code* und die *Umgebung* zum Ausführen der Komponente. Befehl ist der Shellbefehl, um die Komponente auszuführen. Code bezieht sich in der Regel auf ein Quellcodeverzeichnis. „Umgebung“ kann eine AzureML-Umgebung (kuratiert oder vom Kunden erstellt), ein Docker-Image oder eine Conda-Umgebung sein.

Führen Sie die folgenden Zellen aus, um eine YAML-Datei für jede Komponente zu erstellen, die Sie als Pipelineschritt ausführen möchten.

In [ ]:
## Laden der Komponenten

Nachdem Sie die einzelnen Komponenten definiert haben, können Sie nun die Komponenten laden, indem Sie auf die YAML-Dateien verweisen. 

## Erstellen der Pipeline

Nachdem Sie die Komponenten erstellt und geladen haben, können Sie die Pipeline erstellen. Sie setzen die beiden Komponenten zu einer Pipeline zusammen. Zunächst soll die Komponente `prep_data` ausgeführt werden. Die Ausgabe der ersten Komponente sollte die Eingabe der zweiten Komponente `train_decision_tree`sein, die das Modell trainiert.

Die Funktion `diabetes_classification` stellt die vollständige Pipeline dar. Diese Funktion erwartet eine Eingabevariable: `pipeline_job_input`. Während der Einrichtung wurde eine Datenressource erstellt. Sie verwenden die registrierte Datenressource als Pipelineeingabe. 

In [ ]:
Sie können die Konfiguration des Pipelineauftrags abrufen, indem Sie das Objekt `pipeline_job` ausgeben:

In [ ]:
Sie können einen beliebigen Parameter der Pipelineauftragskonfiguration ändern, indem Sie auf den Parameter verweisen und den neuen Wert angeben:

In [ ]:
## Übermitteln des Pipelineauftrags

Wenn Sie schließlich die Pipeline erstellt und den Pipelineauftrag so konfiguriert haben, dass er wie erforderlich ausgeführt wird, können Sie den Pipelineauftrag übermitteln:

## Define the components

To define the component you need to specify:

- **Metadata**: *name*, *display name*, *version*, *description*, *type* etc. The metadata helps to describe and manage the component.
- **Interface**: *inputs* and *outputs*. For example, a model training component will take training data and the regularization rate as input, and generate a trained model file as output. 
- **Command, code & environment**: the *command*, *code* and *environment* to run the component. Command is the shell command to execute the component. Code usually refers to a source code directory. Environment could be an AzureML environment (curated or custom created), docker image or conda environment.

Run the following cells to create a YAML for each component you want to run as a pipeline step.

In [ ]:
%%writefile prep-data.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: prep_data
display_name: Prepare training data
version: 1
type: command
inputs:
  input_data: 
    type: uri_file
outputs:
  output_data:
    type: uri_file
code: ./src
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
command: >-
  python prep-data.py 
  --input_data ${{inputs.input_data}}
  --output_data ${{outputs.output_data}}

In [ ]:
%%writefile train-model.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_model
display_name: Train a decision tree classifier model
version: 1
type: command
inputs:
  training_data: 
    type: uri_file
  reg_rate:
    type: number
    default: 0.01
outputs:
  model_output:
    type: mlflow_model
code: ./src
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
command: >-
  python train-model.py 
  --training_data ${{inputs.training_data}} 
  --reg_rate ${{inputs.reg_rate}} 
  --model_output ${{outputs.model_output}} 

## Load the components

Now that you have defined each component, you can load the components by referring to the YAML files. 

In [ ]:
from azure.ai.ml import load_component
parent_dir = ""

prep_data = load_component(source=parent_dir + "./prep-data.yml")
train_decision_tree = load_component(source=parent_dir + "./train-model.yml")

## Build the pipeline

After creating and loading the components, you can build the pipeline. You'll compose the two components into a pipeline. First, you'll want the `prep_data` component to run. The output of the first component should be the input of the second component `train_decision_tree`, which will train the model.

The `diabetes_classification` function represents the complete pipeline. The function expects one input variable: `pipeline_job_input`. A data asset was created during setup. You'll use the registered data asset as the pipeline input. 

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

@pipeline()
def diabetes_classification(pipeline_job_input):
    clean_data = prep_data(input_data=pipeline_job_input)
    train_model = train_decision_tree(training_data=clean_data.outputs.output_data)

    return {
        "pipeline_job_transformed_data": clean_data.outputs.output_data,
        "pipeline_job_trained_model": train_model.outputs.model_output,
    }

pipeline_job = diabetes_classification(Input(type=AssetTypes.URI_FILE, path="azureml:diabetes-data:1"))

You can retrieve the configuration of the pipeline job by printing the `pipeline_job` object:

In [ ]:
print(pipeline_job)

You can change any parameter of the pipeline job configuration by referring to the parameter and specifying the new value:

In [ ]:
# change the output mode
pipeline_job.outputs.pipeline_job_transformed_data.mode = "upload"
pipeline_job.outputs.pipeline_job_trained_model.mode = "upload"
# set pipeline level compute
pipeline_job.settings.default_compute = "aml-cluster"
# set pipeline level datastore
pipeline_job.settings.default_datastore = "workspaceblobstore"

# print the pipeline job again to review the changes
print(pipeline_job)

## Submit the pipeline job

Finally, when you've built the pipeline and configured the pipeline job to run as required, you can submit the pipeline job:

In [ ]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_diabetes"
)
pipeline_job